In [1]:
import torch
import glob
import os
# glob 결과 숫자 오름차순으로 정리해주는 라이브러리, 기능적으로 필요하지 않았음을 깨달았으나
# 정렬 작업이 유지보수를 가정했을 때 충분히 의미 있다고 생각해서 그냥 놔두기로 함
import natsort
from PIL import Image
from torchvision import transforms
from torch.utils.data import DataLoader
from torch import nn
from torchvision import models
from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter

import numpy as np
import time

In [2]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')
import drive.MyDrive.Colab_Notebooks.resnet_datanmodel as datanmodel
import drive.MyDrive.Colab_Notebooks.ResNet as ResNet

Mounted at /content/drive


In [3]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

In [4]:
# 경로 설정, py파일로 변환시 경로는 변경되어야 한다.
# local path
# path=os.path.abspath('../')
# colab path
path=os.path.abspath('./drive/MyDrive/Colab_Notebooks/')

# Resize: 크기를 224, 224로 맞춘다
# ToTensor: 데이터 타입을 Tensor로 만든다. Tensor의 원소는 0~1로 정해진다.(https://pytorch.org/vision/stable/generated/torchvision.transforms.ToTensor.html#torchvision.transforms.ToTensor)
# custom으로 transform를 작성하는 것도 가능하다.
transforms=transforms.Compose([
    transforms.Resize(size=(224, 224)),
    transforms.ToTensor()])

In [5]:
cnd_test=datanmodel.cnd_data(file_path=path, train=False, transforms=transforms)

In [6]:
batch=32
cnd_dataloader=DataLoader(cnd_test, batch_size=batch, shuffle=True)

In [7]:
# return test_model
# test_model=datanmodel.resnet_model_loader(50, source='git')
# test_model=datanmodel.resnet_model_loader(18)
test_model=datanmodel.resnet_model_loader(34)
# summary(test_model, input_size=(3, 224, 224))
# print(test_model)

In [8]:
loss_f= nn.CrossEntropyLoss()

In [9]:
log_save_path=os.path.abspath('./drive/MyDrive/Colab_Notebooks/resnet/resnet_test_log')

# resnet 18
# weight_save_path=os.path.abspath('./drive/MyDrive/Colab_Notebooks/resnet/resnet_18_pth_log/resnet_pth/')
# resnet 34
# weight_save_path=os.path.abspath('./drive/MyDrive/Colab_Notebooks/resnet/resnet_34_pth_log/resnet_pth/')
# resnet 50
# weight_save_path=os.path.abspath('./drive/MyDrive/Colab_Notebooks/resnet/resnet_50_pth_log/resnet_pth/')
# resnet 50 github
weight_save_path=os.path.abspath('./drive/MyDrive/Colab_Notebooks/resnet/resnet_50_github_pth_log/resnet_pth/')

In [10]:
writer=SummaryWriter(log_save_path)

In [11]:
weight_list=natsort.natsorted(glob.glob(weight_save_path+'/*.pth'), reverse=False)

In [12]:
# 40 epoch까지의 값들을 확인한다.
START_EPOCHS=0
EPOCHS=40
weight_list=weight_list[START_EPOCHS:EPOCHS]

In [13]:
for epoch_path in weight_list:

    print(epoch_path)


# 멀티코어를 사용한 가중치일때 추가로 붙는 key제거하기
    loaded_weight=datanmodel.weight_loader(device, test_model, epoch_path)
    test_model.load_state_dict(loaded_weight)

    epoch=int(epoch_path.split('_')[-1].split('.')[0])


    losses=[]
    accs=[]

    # epoch_total_loss: 1 epoch에서 발생한 loss 누적값, 750(1 epoch batch수)을 나눠서 평균 loss값을 구하는데 사용할 예정
    epoch_total_loss=0
    epoch_total_acc=0
    start_time=time.time()


    for i, inp in enumerate(cnd_dataloader):

        input, label= inp
        input, label= input.to(device), torch.Tensor(label).to(device)


        # test_model을 태운 다음 loss를 계산한다.
        output= test_model(input)
        loss= loss_f(output, label)

        # accuracy를 계산한다.
        correct=0
        for t in zip(output.tolist(), label.tolist()):
          if t[0][0] > t[0][1]:
            ans=0
          else:
            ans=1


          if ans==t[1]:
            correct+=1
          else:
            pass

        # 계산한 loss를 losses에 추가한다.

        epoch_total_loss+=loss.item()
        epoch_total_acc+=correct


    end_time=time.time()

    avg_loss=(epoch_total_loss/100) / len(cnd_test)
    avg_acc=epoch_total_acc / len(cnd_test)

    print('epoch_total_acc',epoch_total_acc)
    print(f'Loss [{epoch+1}](epoch): ', avg_loss)
    print(f'Accuracy [{epoch+1}](epoch): ', avg_acc)
    print('time taken(per epoch):', end_time-start_time)

    losses.append(avg_loss)
    accs.append(avg_acc)

    writer.add_scalar("Loss / epoch ", epoch_total_loss/len(cnd_test), epoch)
    writer.add_scalar("Accuracy / epoch", epoch_total_acc/len(cnd_test), epoch)

writer.close()


/content/drive/MyDrive/Colab_Notebooks/resnet/resnet_34_pth_log/resnet_pth/model_weights_0.pth
gpu 병렬 x
epoch_total_acc 660
Loss [1](epoch):  0.00020000685811042784
Accuracy [1](epoch):  0.66
time taken(per epoch): 785.7873771190643
/content/drive/MyDrive/Colab_Notebooks/resnet/resnet_34_pth_log/resnet_pth/model_weights_1.pth
gpu 병렬 x
epoch_total_acc 720
Loss [2](epoch):  0.0001860804569721222
Accuracy [2](epoch):  0.72
time taken(per epoch): 8.92219066619873
/content/drive/MyDrive/Colab_Notebooks/resnet/resnet_34_pth_log/resnet_pth/model_weights_2.pth
gpu 병렬 x
epoch_total_acc 737
Loss [3](epoch):  0.00018080736368894577
Accuracy [3](epoch):  0.737
time taken(per epoch): 8.38738489151001
/content/drive/MyDrive/Colab_Notebooks/resnet/resnet_34_pth_log/resnet_pth/model_weights_3.pth
gpu 병렬 x
epoch_total_acc 801
Loss [4](epoch):  0.00015984883785247802
Accuracy [4](epoch):  0.801
time taken(per epoch): 8.24784779548645
/content/drive/MyDrive/Colab_Notebooks/resnet/resnet_34_pth_log/resnet